In [1]:
from pref_voting.generate_profiles import *
from pref_voting.profiles_with_ties import *
from pref_voting.voting_methods import *
from pref_voting.rankings import Ranking
from tqdm.notebook import tqdm
from pref_voting.invariance_axioms import preferential_equality, tiebreaking_compensation
from pref_voting.generate_profiles import strict_weak_orders
from pref_voting.variable_voter_axioms import nonlinear_neutral_reversal, neutral_indifference

In [2]:
lin_order = (2, 1, 3)

r = Ranking({0:1, 1:2, 2:2, 3:2, 4:3, 5:3})


print(r.to_indiff_list())

new_indiff_list = []
for cs in r.to_indiff_list():
    if set(cs) == set(lin_order):
        for c in lin_order: 
            new_indiff_list.append((c,))
    else:
        new_indiff_list.append(cs)

print(lin_order)
print(r.is_tied(lin_order))
print(r.is_tied([4, 5]))
print(r.is_tied([0, 1]))
print(r.is_tied([1, 2, 3, 4, 5]))
print(r.is_tied([0]))
print(Ranking.from_indiff_list(new_indiff_list))

((0,), (1, 2, 3), (4, 5))
(2, 1, 3)
True
True
False
False
True
0 2 1 3 ( 4  5 ) 


In [3]:

print(linear_orders_with_reverse([1, 2, 3]))

for r1, reverse_r1 in linear_orders_with_reverse([1, 2, 3]): 
    print(r1, ": ", reverse_r1)

for r1, r2 in combinations([0, 0, 1, 2], 2):
    print(r1, r2)

NameError: name 'linear_orders_with_reverse' is not defined

In [8]:
from itertools import chain, combinations
# generate all subsets of a set, use combinations
def powerset(lst):
    s = list(lst)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def linear_orders_with_reverse(cands): 

    lin_orders = list(permutations(cands))
    lin_orders_with_reverse = []
    for lin_order in lin_orders:
        lin_orders_with_reverse.append((lin_order, lin_order[::-1]))
    return lin_orders_with_reverse

def remove_first_occurrences(rankings, r1, r2):
    removed_r1 = False
    removed_r2 = False
    result = []

    for r in rankings:
        if r == r1 and not removed_r1:
            removed_r1 = True  # Skip the first r1
        elif r == r2 and not removed_r2:
            removed_r2 = True  # Skip the first r2
        else:
            result.append(r)  # Keep all other elements

    return result


prof = generate_profile(3, 10).to_profile_with_ties()


def has_tiebreaking_compensation_violation(prof, vm, verbose=False):
    """
    Return True if the profile prof has a tiebreaking compensation violation for the voting method vm.
    """
    for cands in powerset(prof.candidates): 
        if len(cands) > 1: 
            rankings_with_tie = [r for r in prof.rankings if r.is_tied(cands)]

            checked_rankings = []
            for r1, r2 in combinations(rankings_with_tie, 2):
                if set([r1, r2]) in checked_rankings:
                    continue
                checked_rankings.append(set([r1, r2]))
                for lin_order, reverse_lin_order in linear_orders_with_reverse(cands): 

                    other_rankings = [r for r in prof.rankings if not r in rankings_with_tie]
                    
                    # rankings_with_tie without r1 and r2
                    other_rankings_with_tie = remove_first_occurrences(rankings_with_tie, r1, r2)

                    new_rankings = [r1.break_tie(lin_order),r2.break_tie(reverse_lin_order)] +  other_rankings_with_tie + other_rankings 

                    new_prof = ProfileWithTies(new_rankings, candidates=prof.candidates)
                    if vm(prof) != vm(new_prof): 
                        if verbose: 
                            prof.anonymize().display()
                            print(prof.description())
                            vm.display(prof)

                            print(f"\nAfter breaking the tie between the candidates {cands} in {r1} with {lin_order} and {r2} with {reverse_lin_order}: \n")
                            new_prof.anonymize().display()
                            print(new_prof.description())
                            vm.display(new_prof)
                        return True
    return False




prof  = ProfileWithTies([
    {0:1, 1:1, 2:3, 3:2},
    {0:1, 1:1, 2:3, 3:2},
    {0:1, 1:1, 2:1, 3:2},
    {0:1, 1:1, 2:1, 3:2},
])


for r in prof.rankings: 
    print(r)
    print(r.is_tied([0, 1, 2]))
print()
print(has_tiebreaking_compensation_violation(prof, minimax, verbose=True))



( 0  1 ) 3 2 
False
( 0  1 ) 3 2 
False
( 0  1  2 ) 3 
True
( 0  1  2 ) 3 
True

False


In [2]:
r1 = Ranking({0:-1000, 1:2})

r2 = Ranking({0:1, 1:100})

print(r1 == r2)

print(hash(r1))
print(hash(r2))

print(hash(r1) == hash(r2)) 

print(r1)

print(r1.reverse())

print(Ranking({0:1, 1:1, 2:2, 3:2, 4:100}).reverse())

True
-8661398524813452123
-8661398524813452123
True
0 1 
1 0 
4 ( 2  3 ) ( 0  1 ) 


In [10]:
for r in strict_weak_orders(range(4)):
    print(Ranking.from_indiff_list(r).reverse())

3 2 1 0 
2 3 1 0 
( 2  3 ) 1 0 
3 1 2 0 
1 3 2 0 
( 1  3 ) 2 0 
2 1 3 0 
1 2 3 0 
( 1  2 ) 3 0 
3 ( 1  2 ) 0 
2 ( 1  3 ) 0 
1 ( 2  3 ) 0 
( 1  2  3 ) 0 
3 2 0 1 
2 3 0 1 
( 2  3 ) 0 1 
3 0 2 1 
0 3 2 1 
( 0  3 ) 2 1 
2 0 3 1 
0 2 3 1 
( 0  2 ) 3 1 
3 ( 0  2 ) 1 
2 ( 0  3 ) 1 
0 ( 2  3 ) 1 
( 0  2  3 ) 1 
3 1 0 2 
1 3 0 2 
( 1  3 ) 0 2 
3 0 1 2 
0 3 1 2 
( 0  3 ) 1 2 
1 0 3 2 
0 1 3 2 
( 0  1 ) 3 2 
3 ( 0  1 ) 2 
1 ( 0  3 ) 2 
0 ( 1  3 ) 2 
( 0  1  3 ) 2 
2 1 0 3 
1 2 0 3 
( 1  2 ) 0 3 
2 0 1 3 
0 2 1 3 
( 0  2 ) 1 3 
1 0 2 3 
0 1 2 3 
( 0  1 ) 2 3 
2 ( 0  1 ) 3 
1 ( 0  2 ) 3 
0 ( 1  2 ) 3 
( 0  1  2 ) 3 
3 2 ( 0  1 ) 
2 3 ( 0  1 ) 
( 2  3 ) ( 0  1 ) 
3 1 ( 0  2 ) 
1 3 ( 0  2 ) 
( 1  3 ) ( 0  2 ) 
2 1 ( 0  3 ) 
1 2 ( 0  3 ) 
( 1  2 ) ( 0  3 ) 
3 0 ( 1  2 ) 
0 3 ( 1  2 ) 
( 0  3 ) ( 1  2 ) 
2 0 ( 1  3 ) 
0 2 ( 1  3 ) 
( 0  2 ) ( 1  3 ) 
1 0 ( 2  3 ) 
0 1 ( 2  3 ) 
( 0  1 ) ( 2  3 ) 
3 ( 0  1  2 ) 
2 ( 0  1  3 ) 
1 ( 0  2  3 ) 
0 ( 1  2  3 ) 
( 0  1  2  3 ) 


In [3]:
r = Ranking.from_linear_order((0, 2, 1, 3))
print(r)

r = Ranking.from_indiff_list(((0,), (2, 1), (3,)))
print(r)


0 2 1 3 
0 ( 2  1 ) 3 


In [3]:
def swap_candidates(ranking, c1, c2):
    """
    Swap two candidates in a ranking.
    :param ranking: either a tuple or a list of candidates or a Ranking object
    :param c1: candidate 1
    :param c2: candidate 2
    :return: a new ranking (a tuple) with c1 and c2 swapped
    """

    if isinstance(ranking, Ranking):

        rmap = ranking.rmap

        if c1 not in rmap or c2 not in rmap:
            raise ValueError("One of the candidates is not in the ranking")
        
        # swap the values associated with c1 and c2
        new_rmap = rmap.copy()
        new_rmap[c1], new_rmap[c2] = new_rmap[c2], new_rmap[c1]
        new_ranking = Ranking(new_rmap)
        
    elif isinstance(ranking, (list, tuple)):
        new_ranking = []
        for c in ranking:
            if c == c1:
                new_ranking.append(c2)
            elif c == c2:
                new_ranking.append(c1)
            else:
                new_ranking.append(c)
        new_ranking = tuple(new_ranking)
    return new_ranking



def equal_size_partitions_with_duplicates(lst):
    """
    Generate all partitions of a list into two distinct subsets of equal size, 
    including cases where the input list contains duplicates and elements 
    that do not support ordering (<).
    
    Parameters:
        lst (list): The input list to partition. Must have an even number of elements.
        
    Returns:
        list of tuples: A list of tuples, where each tuple contains two lists of equal size.
    """
    if len(lst) % 2 != 0:
        raise ValueError("The input list must have an even number of elements.")
    
    n = len(lst) // 2
    partitions = []
    
    # Use frozensets to track seen partitions without requiring ordering
    seen = set()
    
    for subset in combinations(lst, n):
        complement = lst[:]
        for item in subset:
            complement.remove(item)
        
        # Use frozensets to handle duplicate tracking, ignoring order
        partition_key = frozenset([frozenset(subset), frozenset(complement)])
        if partition_key not in seen:
            seen.add(partition_key)
            partitions.append((list(subset), complement))
    
    return partitions

# Example usage
lst = [2, 2, 2, 4]
partitions = equal_size_partitions_with_duplicates(lst)
for p in partitions:
    print(p)
for I, J in equal_size_partitions_with_duplicates([(0, 1, 3, 2), (0, 1, 3, 2)]): 
    print(I, J)


for I, J in equal_size_partitions_with_duplicates([Ranking({0:1, 1:2}), Ranking({0:2, 1:20}),  Ranking({0:1, 1:2}), Ranking({0:1, 1:2})]): 
    print(I, J)


def get_rank(ranking, c):
    """
    Get the (normalized) rank of a candidate in a ranking.
    :param ranking: either a tuple or a list of candidates or a Ranking object
    :param c: candidate
    :return: the rank of c in the ranking
    """
    if isinstance(ranking, Ranking):
        norm_ranking = copy.deepcopy(ranking)
        norm_ranking.normalize_ranks()
        return norm_ranking.rmap[c]
    elif isinstance(ranking, (list, tuple)):
        return ranking.index(c)
    else:
        raise ValueError("Invalid input type")

([2, 2], [2, 4])
[(0, 1, 3, 2)] [(0, 1, 3, 2)]
[<pref_voting.rankings.Ranking object at 0x2a2239110>, <pref_voting.rankings.Ranking object at 0x2a2239190>] [<pref_voting.rankings.Ranking object at 0x2a2239250>, <pref_voting.rankings.Ranking object at 0x2a2239310>]


In [25]:


def has_preferential_equality_violation(prof, vm, verbose=False):
    """
    Check if a profile has a preferential equality violation for the voting method vm.

    See Definition 2.1 and Lemma 2.4 from the paper "Characterizations of voting rules based on majority margins" by Y. Ding,  W. Holliday, and E. Pacuit.

    """
    prof_constructor = ProfileWithTies if isinstance(prof, ProfileWithTies) else Profile
    
    for x, y in combinations(prof.candidates, 2):
        
        xy_rankings = [r for r in prof.rankings if get_rank(r, x) == get_rank(r, y) - 1]

        other_rankings = [r for r in prof.rankings if r not in xy_rankings]

        if len(xy_rankings) != 0 and len(xy_rankings) % 2 == 0:
            for I, J in equal_size_partitions_with_duplicates(xy_rankings):
                new_rankings_I = [swap_candidates(r, x, y) for r in I] + list(J) + list(other_rankings)
                prof_I = prof_constructor(new_rankings_I)
                new_rankings_J = [swap_candidates(r, x, y) for r in J] + list(I) + list(other_rankings)
                 
                prof_J = prof_constructor(new_rankings_J)

                if vm(prof_I) != vm(prof_J): 
                    if verbose:
                        print("The original profile")
                        prof.anonymize().display()
                        print(prof.description())
                        print(f"The profile after swapping {x} and {y} in the rankings {I}:")
                        prof_I.anonymize().display()
                        print(prof_I.description())
                        vm.display(prof_I)
                        print(f"The profile after swapping {x} and {y} in the rankings {J}:")
                        prof_J.anonymize().display()
                        print(prof_J.description())
                        vm.display(prof_J)
                    return True

        yx_rankings = [r for r in prof.rankings if get_rank(r, y) == get_rank(r, x) - 1]

        other_rankings = [r for r in prof.rankings if r not in yx_rankings]

        if len(yx_rankings) != 0 and len(yx_rankings) % 2 == 0:
            for I, J in equal_size_partitions_with_duplicates(yx_rankings):
                new_rankings_I = [swap_candidates(r, y, x) for r in I] + list(J) + list(other_rankings)
                prof_I = prof_constructor(new_rankings_I)
                new_rankings_J = [swap_candidates(r, y, x) for r in J] + list(I) + list(other_rankings)
                prof_J = prof_constructor(new_rankings_J)

                if vm(prof_I) != vm(prof_J): 
                    if verbose:
                        print("The original profile")
                        prof.anonymize().display()
                        print(prof.description())
                        print(f"The profile after swapping {x} and {y} in the rankings {I}:")
                        prof_I.anonymize().display()
                        print(prof_I.description())
                        vm.display(prof_I)
                        print(f"The profile after swapping {x} and {y} in the rankings {J}:")
                        prof_J.anonymize().display()
                        print(prof_J.description())
                        vm.display(prof_J)
                    return True
    return False


def find_all_preferential_equality_violations(prof, vm, verbose=False):
    """
    Return all the preferential equality violations for the voting method vm.  Returns a list of tuples of three profiles (prof, prof_I, prof_J) such that vm(prof_I) != vm(prof_J) and prof_I and prof_J are as defined Lemma 2.4 from the paper "Characterizations of voting rules based on majority margins" by Y. Ding,  W. Holliday, and E. Pacuit (see also Definition 2.1).

    """

    prof_constructor = ProfileWithTies if isinstance(prof, ProfileWithTies) else Profile
    
    violations = []
    for x, y in combinations(prof.candidates, 2):
        
        xy_rankings = [r for r in prof.rankings if get_rank(r, x) == get_rank(r, y) - 1]

        other_rankings = [r for r in prof.rankings if r not in xy_rankings]

        if len(xy_rankings) != 0 and len(xy_rankings) % 2 == 0:
            for I, J in equal_size_partitions_with_duplicates(xy_rankings):
                new_rankings_I = [swap_candidates(r, x, y) for r in I] + list(J) + list(other_rankings)
                prof_I = prof_constructor(new_rankings_I)
                new_rankings_J = [swap_candidates(r, x, y) for r in J] + list(I) + list(other_rankings)
                 
                prof_J = prof_constructor(new_rankings_J)

                if vm(prof_I) != vm(prof_J): 
                    if verbose:
                        print("The original profile")
                        prof.anonymize().display()
                        print(prof.description())
                        print(f"The profile after swapping {x} and {y} in the rankings {I}:")
                        prof_I.anonymize().display()
                        print(prof_I.description())
                        vm.display(prof_I)
                        print(f"The profile after swapping {x} and {y} in the rankings {J}:")
                        prof_J.anonymize().display()
                        print(prof_J.description())
                        vm.display(prof_J)
                    violations.append((prof, prof_I, prof_J))

        yx_rankings = [r for r in prof.rankings if get_rank(r, y) == get_rank(r, x) - 1]

        other_rankings = [r for r in prof.rankings if r not in yx_rankings]

        if len(yx_rankings) != 0 and len(yx_rankings) % 2 == 0:
            for I, J in equal_size_partitions_with_duplicates(yx_rankings):
                new_rankings_I = [swap_candidates(r, y, x) for r in I] + list(J) + list(other_rankings)
                prof_I = prof_constructor(new_rankings_I)
                new_rankings_J = [swap_candidates(r, y, x) for r in J] + list(I) + list(other_rankings)
                prof_J = prof_constructor(new_rankings_J)

                if vm(prof_I) != vm(prof_J): 
                    if verbose:
                        print("The original profile")
                        prof.anonymize().display()
                        print(prof.description())
                        print(f"The profile after swapping {x} and {y} in the rankings {I}:")
                        prof_I.anonymize().display()
                        print(prof_I.description())
                        vm.display(prof_I)
                        print(f"The profile after swapping {x} and {y} in the rankings {J}:")
                        prof_J.anonymize().display()
                        print(prof_J.description())
                        vm.display(prof_J)
                    violations.append((prof, prof_I, prof_J))
    return violations



In [3]:
profiles = [generate_profile(5, 11).to_profile_with_ties() for _ in tqdm(range(1000))]


  0%|          | 0/1000 [00:00<?, ?it/s]

In [28]:
for prof in profiles: 
    print(find_all_preferential_equality_violations(prof, vm, verbose=False))

[]
[(<pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e5c074d0>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e6053dd0>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e35bd090>)]
[(<pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e5bfb610>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e35e53d0>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e3568e90>)]
[]
[]
[]
[(<pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e5bfae10>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e34cfb90>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e34cf6d0>)]
[]
[]
[]
[]
[]
[(<pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e5be53d0>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e2f5bf90>, <pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e35010d0>)]
[(<pref_voting.profiles_with_ties.ProfileWithTies object at 0x2e5be7c90>, <pref_voting.profiles_w

In [14]:

prof2s = [generate_profile(5, 11) for _ in tqdm(range(1000))]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
%%timeit

np.mean([preferential_equality.has_violation(prof, instant_runoff, verbose=False) for prof in profiles])

18 s ± 258 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit

np.mean([has_preferential_equality_violation(prof, instant_runoff_for_truncated_linear_orders, verbose=False) for prof in profs])

17.7 s ± 87.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
def has_neutral_indifference_violation(prof, vm, verbose=False): 
    """
    Return True if the profile prof has a neutral indifference violation for the voting method vm.  Otherwise, return False.  That is, return True if there is a tie ranking that can be added to the profile that changes winning set according to vm.  Otherwise, return False.
    """

    tie_ranking = Ranking({c:0 for c in prof.candidates})
    if isinstance(prof, ProfileWithTies):
        new_rankings = prof.rankings + [tie_ranking]
        new_prof = ProfileWithTies(new_rankings)
    elif isinstance(prof, Profile):
        new_rankings = [Ranking.from_linear_order(r) for r in prof.rankings] + [tie_ranking]
        new_prof = ProfileWithTies(new_rankings)
    if vm(prof) != vm(new_prof):
        if verbose:
            print("The original profile")
            prof.anonymize().display()
            print(prof.description())
            vm.display(prof)
            print(f"The profile after adding a tie ranking:")
            new_prof.anonymize().display()
            print(new_prof.description())
            vm.display(new_prof)
        return True
    return False

def find_all_neutral_indifference_violations(prof, vm, verbose=False): 
    """
    Return a list containing the profile with an additional voter that ranks all candidates as a tie if this profile has a different winning set according to vm than the original profile.  Otherwise, return the empty list.
    """

    tie_ranking = Ranking({c:0 for c in prof.candidates})
    if isinstance(prof, ProfileWithTies):
        new_rankings = prof.rankings + [tie_ranking]
        new_prof = ProfileWithTies(new_rankings)
    elif isinstance(prof, Profile):
        new_rankings = [Ranking.from_linear_order(r) for r in prof.rankings] + [tie_ranking]
        new_prof = ProfileWithTies(new_rankings)
    if vm(prof) != vm(new_prof):
        if verbose:
            print("The original profile")
            prof.anonymize().display()
            print(prof.description())
            vm.display(prof)
            print(f"The profile after adding a tie ranking:")
            new_prof.anonymize().display()
            print(new_prof.description())
            vm.display(new_prof)
        return [new_prof]
    return []

In [15]:
for t in range(1000):
    prof = generate_profile(3, 11)

    if find_all_neutral_indifference_violations(prof, pareto, verbose=True): 
        print("FOUND IT!")
        prof.anonymize().display()

The original profile
+---+---+---+
| 6 | 2 | 3 |
+---+---+---+
| 0 | 2 | 2 |
| 2 | 1 | 0 |
| 1 | 0 | 1 |
+---+---+---+
Profile([[0, 2, 1], [0, 2, 1], [2, 1, 0], [2, 0, 1], [0, 2, 1], [2, 0, 1], [0, 2, 1], [0, 2, 1], [2, 1, 0], [0, 2, 1], [2, 0, 1]], rcounts=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], cmap={0: '0', 1: '1', 2: '2'})
Pareto winners are {0, 2}
The profile after adding a tie ranking:
+---+---+---+-------+
| 6 | 2 | 3 |   1   |
+---+---+---+-------+
| 0 | 2 | 2 | 0 1 2 |
| 2 | 1 | 0 |       |
| 1 | 0 | 1 |       |
+---+---+---+-------+
ProfileWithTies([{0: 1, 2: 2, 1: 3}, {0: 1, 2: 2, 1: 3}, {2: 1, 1: 2, 0: 3}, {2: 1, 0: 2, 1: 3}, {0: 1, 2: 2, 1: 3}, {2: 1, 0: 2, 1: 3}, {0: 1, 2: 2, 1: 3}, {0: 1, 2: 2, 1: 3}, {2: 1, 1: 2, 0: 3}, {0: 1, 2: 2, 1: 3}, {2: 1, 0: 2, 1: 3}, {0: 0, 1: 0, 2: 0}], rcounts=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], cmap={0: 0, 1: 1, 2: 2})
Pareto winners are {0, 1, 2}
FOUND IT!
+---+---+---+
| 6 | 2 | 3 |
+---+---+---+
| 0 | 2 | 2 |
| 2 | 1 | 0 |
| 1 | 0 | 1 |
+---

In [12]:
def has_nonlinear_neutral_reversal_violation(prof, vm, verbose=False): 
    """
    Return True if there is a violation of the nonlinear neutral reversal axiom for the voting method vm.  Otherwise, return False.  That is, return True if there is a strict weak order and its reverse that can be added to the profile that changes the winning set according to vm.  
    """
    for swo in strict_weak_orders(prof.candidates):

        ranking = Ranking.from_indiff_list(swo)
        ranking_reverse = ranking.reverse()

        if isinstance(prof, ProfileWithTies):
            new_rankings = prof.rankings + [ranking, ranking_reverse]
            new_prof = ProfileWithTies(new_rankings)

        elif isinstance(prof, Profile):
            new_rankings = [Ranking.from_linear_order(r) for r in prof.rankings] + [ranking, ranking_reverse]
            new_prof = ProfileWithTies(new_rankings)
        
        if vm(prof) != vm(new_prof):
            if verbose:
                print("The original profile")
                prof.anonymize().display()
                print(prof.description())
                vm.display(prof)
                print(f"The profile after adding {ranking} and its reverse {ranking_reverse}:")
                new_prof.anonymize().display()
                print(new_prof.description())
                vm.display(new_prof)
            return True
    return False


def find_all_nonlinear_neutral_reversal_violations(prof, vm, verbose=False): 
    """
    Return the list of strict weak orders and their reverse such that adding them to prof results in a different winning set according to vm.  Otherwise, return the empty list.  
    """

    violation_swos = []
    for swo in strict_weak_orders(prof.candidates):

        ranking = Ranking.from_indiff_list(swo)
        ranking_reverse = ranking.reverse()

        if isinstance(prof, ProfileWithTies):
            new_rankings = prof.rankings + [ranking, ranking_reverse]
            new_prof = ProfileWithTies(new_rankings)

        elif isinstance(prof, Profile):
            new_rankings = [Ranking.from_linear_order(r) for r in prof.rankings] + [ranking, ranking_reverse]
            new_prof = ProfileWithTies(new_rankings)
        
        if vm(prof) != vm(new_prof):
            if verbose:
                print("The original profile")
                prof.anonymize().display()
                print(prof.description())
                vm.display(prof)
                print(f"The profile after adding {ranking} and its reverse {ranking_reverse}:")
                new_prof.anonymize().display()
                print(new_prof.description())
                vm.display(new_prof)
            violation_swos.append((ranking, ranking_reverse))

    return violation_swos


In [5]:
for t in range(1000): 
    prof = generate_profile(5, 11)
    if nonlinear_neutral_reversal.find_all_violations(prof.to_profile_with_ties(), borda, verbose=True):
        print("FOUND IT!")
        prof.anonymize().display()
